In [1]:
import math
import numpy as np

In [2]:
def K(s, t, n, decay):
#     s, t - stringi
#     n - długość podciągów
#     decay = lambda 
#     zwraca wartość K_{n}(s,t)
    S = np.zeros((n, len(s)+1, len(t)+1))
    k = 0
    
    if n == 1:
        for i in range(0, len(s)):
            for j in range(0, len(t)):
                if s[i] == t[j]:
                     k += decay ** 2
        return k
                
    for l in range(1, n):
        for i in range(0,len(s)):
            for j in range(0, len(t)):
                S[l, i+1, j+1] = decay * S[l, i, j+1] + decay * S[l, i+1, j] - (decay ** 2) * S[l, i, j]
                if s[i] == t[j]:
                    if l == 1:
                        S[l, i+1, j+1] += (decay ** 2)
                    else:
                        S[l, i+1, j+1] += (decay ** 2) * S[l-1, i, j]
                    if l == (n-1):
                        k += S[l, i, j]

    return k * (decay**2)

In [4]:
def GramMatrix(n, decay, X, Y = None):
#     argumenty:
#         n - długość podciągów
#         decay - lambda
#         X - tablica danych
#         Y - tablica danych
#     Funkcja zwraca macierz Grama 
    symetric = False
    K_X = np.array([K(x, x, n, decay) for x in X])
    if Y is None:
        symetric = True
        Y = X
        K_Y = K_X
    else:
        K_Y = np.array([K(x, x, n, decay) for x in Y]) 
        
    matrix = np.zeros((len(X), len(Y)))
    for i in range(len(X)):
        for j in range(len(Y)):
            if symetric and i > j:
                continue
            t = K_X[i]*K_Y[j]
            denom = math.sqrt(t) if t else 10e-30
            matrix[i, j] = K(X[i], Y[j], n, decay) / denom
            
            if symetric:
                matrix[j, i] = matrix[i, j]
        
    return matrix

In [7]:
def top_nsub(data, n):
#     funkcja do aproksymacji jądra
#     zwraca listę posortowanych malejąco najczęściej występujących podciągów długości n
    nsub_count = dict()
    for string in data:
        for i in range(len(string) - n + 1):
            sub = string[i:(i+n)]
            if sub in nsub_count.keys():
                nsub_count[sub] += 1
            else:
                nsub_count[sub] = 1
    return sorted(nsub_count, key = lambda t: nsub_count[t], reverse = True)


def sub_K(data, strings, n, decay):
#     argumenty:
#         data - tablica stringów
#         strings - docelowo podzbiór wyniku funkcji top_nsub
#         n - długość podciągów
#         decay - lambda
# 
#     zwraca macierz wielkości len(data) x len(strings),
#     zawierającą na pozycji (i,j) znormalizowaną wartość ssk dla data[i] oraz strings[j]
    sub = np.zeros((len(data),len(strings)))
    data_self = np.array([K(x, x, n, decay) for x in data])
    strings_self = np.array([K(x, x, n, decay) for x in strings])
    
    for i in range(len(data)):
        sub[i, :] = [K(data[i], x, n, decay) for x in strings]
    
        for j in range(len(strings)):
            t = data_self[i] * strings_self[j]
            denom = math.sqrt(t) if t else 10e-30
            sub[i, j] /= denom
    
    return sub


def Gram_aprox(n, decay, strings, X, Y = None):
#     funkcja liczy aproksymację macierzy Grama dla ssk
#     argumenty:
#         n - długość podciągów
#         decay - lambda
#         strings - podzbiór wyniku funkcji top_nsub
#         X - tablica stringów
#         Y - tablica stringów
    if Y is None:
        lx = len(X)
        matrix = np.zeros((lx, lx))
        sub = sub_K(X, strings, n, decay) 
        for i in range(lx):
            for j in range(lx):
                if i > j:
                    continue
                il = sum(sub[i]**2)*sum(sub[j]**2)
                denom = math.sqrt(il) if il else 10e-30
                matrix[i, j] = sum(sub[i]*sub[j]) / denom
                matrix[j, i] = matrix[i, j]
    else:
        lx = len(X)
        matrix = np.zeros((lx, len(Y)))
        sub = sub_K(np.concatenate((X, Y)), strings, n, decay)
        for i in range(lx):
            for j in range(len(Y)):
                il = sum(sub[i]**2)*sum(sub[(lx + j)]**2)
                denom = math.sqrt(il) if il else 10e-20
                matrix[i, j] = sum(sub[i]*sub[(lx + j)]) / denom
            
    return matrix

In [6]:
def aligment(k1,k2):
#     oblicza podobieństwo pomiędzy jądrami k1, k2 (policzonymi na pewnych danych)
    l = np.sum(np.multiply(k1,k2))
    d1 = math.sqrt(np.sum(np.multiply(k1,k1))*np.sum(np.multiply(k2,k2)))
    d = d1 if d1 else 10e-30
    return l/d